In [74]:
#!usr/bin/env python3
#! -*- coding: utf-8 -*-
#xiaobaodas version_alpha_01 2017-03-13
def xiaobaods_w01(date="",category="牛仔裤",length=7,SQL="xiaobaods",choice="热搜核心词",variable="排名",fillna="",debug=0,path=""):
    import time
    time_s = time.time()
    import sys
    import pandas as pd
    import datetime
    import pymysql
    from dateutil.parser import parse
    SQL_msg = {"local":{"host":"127.0.0.1","port":3306,"user":"root","charset":"utf8","passwd":"","db":"baoersqlbasic"},
               "xiaobaods":{"host":"101.201.237.58","port":3306,"user":"program_default","charset":"utf8","passwd":"KQPp5wwZJG33fwFs","db":"baoersqlbasic"}}
    latest_date=datetime.datetime.today().date()-datetime.timedelta(1)
    choice_list = {"热搜修饰词":{"table":"bc_searchwords_hotwords","variable":("搜索人气","相关搜索词数","点击率","点击人气","支付转化率","直通车参考价")},
                  "热搜品牌词":{"table":"bc_searchwords_hotwords","variable":("搜索人气","相关搜索词数","点击率","点击人气","支付转化率","直通车参考价")},
                  "热搜搜索词":{"table":"bc_searchwords_hotwords","variable":("搜索人气","商城点击占比","点击率","点击人气","支付转化率","直通车参考价")},
                  "热搜核心词":{"table":"bc_searchwords_hotwords","variable":("搜索人气","相关搜索词数","点击率","点击人气","支付转化率","直通车参考价")},
                  "热搜长尾词":{"table":"bc_searchwords_hotwords","variable":("搜索人气","商城点击占比","点击率","点击人气","支付转化率","直通车参考价")},
                  "飙升修饰词":{"table":"bc_searchwords_risewords",
                           "variable":("相关搜索词数","搜索人气","词均搜索增长幅度","点击人气","支付转化率","直通车参考价")},
                  "飙升品牌词":{"table":"bc_searchwords_risewords",
                           "variable":("相关搜索词数","搜索人气","词均搜索增长幅度","点击人气","支付转化率","直通车参考价")},
                  "飙升搜索词":{"table":"bc_searchwords_risewords",
                           "variable":("搜索增长幅度","搜索人气","点击率","点击人气","支付转化率","直通车参考价")},
                  "飙升核心词":{"table":"bc_searchwords_risewords",
                           "variable":("相关搜索词数","搜索人气","词均搜索增长幅度","点击人气","支付转化率","直通车参考价")},
                  "飙升长尾词":{"table":"bc_searchwords_risewords",
                           "variable":("搜索增长幅度","搜索人气","点击率","点击人气","支付转化率","直通车参考价")},}
    if category not in ["牛仔裤","打底裤","休闲裤"]:
        category="牛仔裤"
    if length>30 or length<3:
        length=7
    if SQL not in SQL_msg:
        SQL=SQL_msg[0]
    if choice not in choice_list:
        choice = "热搜修饰词"
    if variable not in choice_list[choice]["variable"]:
        variable="排名"
    if date=="":
        date = latest_date
    else:
        date=parse(date).date()  # 修改日期格式
    # Try to connect with the mysql and back a date which minimum.
    try:
        conn = pymysql.connect(host=SQL_msg[SQL]["host"], port=SQL_msg[SQL]["port"], user=SQL_msg[SQL]["user"],passwd=SQL_msg[SQL]["passwd"],
                               charset=SQL_msg[SQL]["charset"], db=SQL_msg[SQL]["db"])
        cursor = conn.cursor()
        cursor.execute("SELECT min(`日期`),max(`日期`) from "+choice_list[choice]["table"]+" where `类目`='"+category+"' and `字段`='"+choice+"';")
        date_limit = cursor.fetchall()
        date_floor = date_limit[0][0]
        date_ceiling = date_limit[0][1] 
        cursor.close()
        conn.close()
    except Exception as e:
        return e
    if date > latest_date:
        date = latest_date
    if date > date_ceiling:
        date = date_ceiling
    if date < date_floor + datetime.timedelta(length-1):
        date = date_floor + datetime.timedelta(length-1)
    # Main program.
    sql_select_f="SELECT CT.`排名`,CT.`搜索词`"
    for i in range(len(choice_list[choice]["variable"])):
        sql_select_f += ",CT.`"+choice_list[choice]["variable"][i]+"`"
    sql_select_m=""
    for i in range(length):
        sql_select_m += ",MAX(CASE ST.日期 WHEN "+str(date - datetime.timedelta(length-i-1)).replace("-","")+" THEN ST."+variable+" ELSE NULL END) AS `日期："+str(date - datetime.timedelta(length-i-1)).replace("-","")+"` "
    sql_select_e="FROM "+choice_list[choice]["table"]+" AS CT LEFT JOIN "+choice_list[choice]["table"]+" AS ST ON CT.搜索词 = ST.搜索词 WHERE CT.`日期` = "+str(date).replace("-","")+" AND CT.类目 = '"+category+"' AND CT.字段='"+choice+"' AND ST.字段='"+choice+"' AND ST.日期 >= "+str(date - datetime.timedelta(length)).replace("-","")+" AND ST.类目 = '"+category+"' GROUP BY CT.`"+variable+"` ORDER BY CT.`排名`;"
    # read msg from Mysql
    conn = pymysql.connect(host=SQL_msg[SQL]["host"], port=SQL_msg[SQL]["port"], user=SQL_msg[SQL]["user"], passwd=SQL_msg[SQL]["passwd"], charset=SQL_msg[SQL]["charset"], db=SQL_msg[SQL]["db"])
    df = pd.io.sql.read_sql_query(sql_select_f+sql_select_m+sql_select_e,conn)
    conn.close()
    df = df.fillna(fillna)
    if debug not in [8,9]:
        print(df.to_json(orient="index"))
    elif debug== 8:
        print ("- Running time：%.4f s"%(time.time()-time_s))
        return df
    elif debug==9:
        import os
        print ("- Running time：%.4f s"%(time.time()-time_s))
        path_default=os.path.join(os.path.expanduser("~"), 'Desktop')
        if not os.path.isdir(path):
            path = path_default
        csv_filename="【数据组】["+choice_list[choice]["table"].split("_")[-1]+"_"+category+"_"+choice+"]"+variable+"_"+datetime.datetime.strftime(date,"%m%d")+"-"+str(length)+"_"+SQL+".csv"
        try:
            df.to_csv(path+"\\"+csv_filename)
            print("> 输出CSV文件：",path,",",csv_filename)
        except Exception as e:
            print("> 输出CSV文件失败，错误原因：",e)
    if __name__ == "__main__":
        try:
            argv = sys.argv[1]
            argv = eval(argv)
        except:
            argv ={}

In [79]:
xiaobaods_w01(debug=9,length=30,fillna=600)

- Running time：0.2792 s
> 输出CSV文件： C:\Users\Administrator\Desktop , 【数据组】[hotwords_牛仔裤_热搜核心词]排名_0316-30_xiaobaods.csv
